In [85]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Inisialisasi Data

In [86]:
m1 = float(input("Massa Pendulum 1: "))
m2 = float(input("Massa Pendulum 2: "))
L1 = float(input("Panjang Tali 1: "))
L2 = float(input("Panjang Tali 2: "))
teta1 = float(input("Sudut Pendulum 1: "))
teta2 = float(input("Sudut Pendulum 2: "))
dt = float(input("Step Size: "))

teta = np.array([[teta1*np.pi/180], [teta2*np.pi/180]])
tetadot = np.array([[0], [0]])
tau = np.array([[0], [0]])
i1 = m1*(L1**2)
i2 = m2*(L2**2)

## Posisi Garis dan Pendulum

In [87]:
def pos():
    global L1, L2, teta
    pos_garis = np.array([[0, 0],
                          [L1*np.sin(teta[0, 0]), (-L1*np.cos(teta[0, 0]))],
                          [L1*np.sin(teta[0, 0])+L2*np.sin(teta[1, 0]), -(L1*np.cos(teta[0, 0])+L2*np.cos(teta[1, 0]))]])
    pos_massa = np.array([[L1*np.sin(teta[0, 0])/2, -L1*np.cos(teta[0, 0])/2], 
                          [L1*np.sin(teta[0, 0])+L2*np.sin(teta[1, 0])/2, -(L1*np.cos(teta[0, 0])+1/2*L2*np.cos(teta[1, 0]))]])
    return pos_garis, pos_massa


## Fungsi Pendulum

In [88]:
def inverse(m):
    determinant = m[0, 0]*m[1, 1]-m[0, 1]*m[1, 0]
    return np.array([[m[1, 1], -1*m[0, 1]], [-1*m[1, 0], m[0, 0]]])/determinant


def pendulum(teta, tetadot):
    global m1, m2, L1, L2, i1, i2, tau
    g = 9.8

    M = np.array([[(m1/4+m2)*(L1**2)+i1, m2*L1*L2*np.cos(teta[0, 0]-teta[1, 0])/2],
                 [m2*L1*L2*np.cos(teta[0, 0]-teta[1, 0])/2, m2*(L2**2)/4+i2]])
    Minv = inverse(M)

    matrix1 = np.matmul(Minv, tau)

    C = np.array([[m2*L1*L2*np.sin(teta[0, 0]-teta[1, 0])*tetadot[1, 0]/2, -1*m2*L1*L2*np.sin(teta[0, 0]-teta[1, 0])*(tetadot[0, 0]-tetadot[1, 0])/2],
                 [-1*m2*L1*L2*np.sin(teta[0, 0]-teta[1, 0])*(tetadot[0, 0]-tetadot[1, 0])/2, -1*m2*L1*L2*np.sin(teta[0, 0]-teta[1, 0])*tetadot[0, 0]/2]])
    A = np.matmul(Minv, C)
    matrix2 = np.matmul(A, tetadot)

    G = np.array([[((m1/2)+m2)*g*L1*np.sin(teta[0, 0])],
                  [m2*g*L2*np.sin(teta[1, 0])/2]])
    matrix3 = np.matmul(Minv, G)

    tetadotdot = matrix1-matrix2-matrix3

    return tetadotdot


## Fungsi Runge Kutta

In [89]:
def rungekuttaupdate():
    global teta, tetadot,  dt

    tetadotdot= pendulum(teta, tetadot)
    k1 = dt/2*tetadotdot
    k2 = dt/2*pendulum(teta+(dt/2)*(tetadot+(k1/2)), tetadot+k1)
    k3 = dt/2*pendulum(teta+(dt/2)*(tetadot+k1/2), tetadot+k2)
    k4 = dt/2*pendulum(teta+dt*(tetadot+k3), tetadot+2*k3)
    teta = teta + dt*(tetadot+((k1+k2+k3)/3))
    tetadot = tetadot + (k1+(2*k2)+(2*k3)+k4)/3


## Animasi

In [92]:
teta = np.array([[teta1*np.pi/180], [teta2*np.pi/180]])
tetadot = np.array([[0], [0]])

%matplotlib

fig = plt.figure(figsize=(10, 10), constrained_layout=True)
gs = fig.add_gridspec(6, 8)

ax_pendulum = fig.add_subplot(gs[:, :4])
ax_teta1 = fig.add_subplot(gs[0:2, 4:])
ax_teta2 = fig.add_subplot(gs[2:4, 4:])
ax_tetadot1 = fig.add_subplot(gs[4:6, 4:6])
ax_tetadot2 = fig.add_subplot(gs[4:6, 6:8])

ax_pendulum.set_title('Pendulum')
ax_teta1.set_title('Teta 1')
ax_teta2.set_title('Teta 2')
ax_tetadot1.set_title('Tetadot 1 Terhadap Teta 1')
ax_tetadot2.set_title('Tetadot 2 Terhadap Teta 2')

ax_teta1.set_xlabel("t (s)")
ax_teta2.set_xlabel("t (s)")
ax_teta1.set_ylabel("Derajat (Radian)")
ax_teta2.set_ylabel("Derajat (Radian)")

pos_garis, pos_massa = pos()

m_plot, = ax_pendulum.plot(pos_massa[:, 0], pos_massa[:, 1], 'bo', zorder=10)
lin_plot, = ax_pendulum.plot(pos_garis[:, 0], pos_garis[:, 1], '-', zorder=5)
trail_plot, = ax_pendulum.plot([], [], 'co', zorder=0, alpha=0.1)

teta1_plot, = ax_teta1.plot([], [], '-')
teta2_plot, = ax_teta2.plot([], [], '-')
tetadot1_plot, = ax_tetadot1.plot([], [], '-')
tetadot2_plot, = ax_tetadot2.plot([], [], '-')

teta1_plot_list = [teta[0, 0]]
teta2_plot_list = [teta[1, 0]]
tetadot1_plot_list = [tetadot[0, 0]]
tetadot2_plot_list = [tetadot[1, 0]]
trail_list = np.array([pos_massa[0, :], pos_massa[1, :]])


def init():
    global L1, L2
    ax_pendulum.set_xlim(-(L1+L2+0.1), (L1+L2+0.1))
    ax_pendulum.set_ylim(-(L1+L2+0.1), (L1+L2+0.1))
    ax_teta1.set_xlim(0, dt*10)
    ax_teta1.set_ylim(-teta[0, 0], teta[0, 0])
    ax_teta2.set_xlim(0, dt*10)
    ax_teta2.set_ylim(-teta[1, 0], teta[1, 0])
    ax_tetadot1.set_xlim(-np.pi, np.pi)
    ax_tetadot1.set_ylim(-np.pi, np.pi)
    ax_tetadot2.set_xlim(-np.pi, np.pi)
    ax_tetadot2.set_ylim(-np.pi, np.pi)

    return m_plot, lin_plot, teta1_plot, teta2_plot, tetadot1_plot, tetadot2_plot, trail_plot,


def plotanimation(step):
    global teta1_plot_list, teta2_plot_list, tetadot1_plot_list, tetadot2_plot_list, trail_list

    for i in range(1):
        rungekuttaupdate()

    pos_garis, pos_massa = pos()

    teta1_plot_list.append(teta[0, 0])
    teta2_plot_list.append(teta[1, 0])
    tetadot1_plot_list.append(tetadot[0, 0])
    tetadot2_plot_list.append(tetadot[1, 0])

    trail_list = np.vstack((trail_list, pos_massa))
    
    ax_teta1.set_xlim(0, len(teta1_plot_list)*dt+dt)
    ax_teta1.set_ylim(min(teta1_plot_list), max(teta1_plot_list))

    ax_teta2.set_xlim(0, len(teta2_plot_list)*dt+dt)
    ax_teta2.set_ylim(min(teta2_plot_list), max(teta2_plot_list))

    m_plot.set_data(pos_massa[:, 0], pos_massa[:, 1])
    lin_plot.set_data(pos_garis[:, 0], pos_garis[:, 1])
    trail_plot.set_data(trail_list[:, 0], trail_list[:, 1])
    teta1_plot.set_data(
        np.arange(0, len(teta1_plot_list)*dt, dt), teta1_plot_list)
    teta2_plot.set_data(
        np.arange(0, len(teta2_plot_list)*dt, dt), teta2_plot_list)
    
    tetadot1_plot.set_data(teta1_plot_list, tetadot1_plot_list)
    tetadot2_plot.set_data(teta2_plot_list, tetadot2_plot_list)

    return m_plot, lin_plot, teta1_plot, teta2_plot, tetadot1_plot, tetadot2_plot, trail_plot,


ani = animation.FuncAnimation(fig, plotanimation, frames=np.arange(
    0, 100, dt), interval=0, init_func=init, blit=False, repeat=False)

plt.show


Using matplotlib backend: Qt5Agg


<function matplotlib.pyplot.show(*, block=None)>

Traceback (most recent call last):
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\backends\backend_qt5.py", line 480, in _draw_idle
    self.draw()
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 407, in draw
    self.figure.draw(self.renderer)
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\artist.py", line 41, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\figure.py", line 1863, in draw
    mimage._draw_list_compositing_images(
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\image.py", line 131, in _draw_list_compositing_images
    a.draw(renderer)
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\artist.py", line 41, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\Users\adity\anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py", line 411, in wrapper
    return func(*